Como no estoy haciendo feature engineering capaz no lo preciso

In [1]:
!pip install imbalanced-learn


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np

from imblearn.under_sampling import RandomUnderSampler

Paso los 2 datasets al disco local de la VM para que los lea mas rapido

In [ ]:
!gsutil cp /home/eanegrin/buckets/b1/datasets/competencia_02_inflacion_adj_cpe.parquet /home/eanegrin/datasets/
!gsutil cp /home/eanegrin/buckets/b1/datasets/competencia_02_inflacion_adj_ipc.parquet /home/eanegrin/datasets/

Copying file:///home/eanegrin/buckets/b1/datasets/competencia_02_fe_v01.parquet...
- [1 files][  7.1 GiB/  7.1 GiB]                                                
Operation completed over 1 objects/7.1 GiB.                                      


In [ ]:
base_path = '/home/eanegrin/buckets/b1/'

dataset_path = base_path + 'datasets/'

dataset_ipc_file = 'competencia_02_inflacion_adj_ipc.parquet'
dataset_cpe_file = 'competencia_02_inflacion_adj_cpe.parquet'

semillas = [122219, 109279, 400391, 401537, 999961]

In [ ]:
df_ipc = pd.read_parquet('/home/eanegrin/datasets/' + dataset_ipc_file)
df_cpe = pd.read_parquet('/home/eanegrin/datasets/' + dataset_cpe_file)

In [ ]:
df_ipc['clase_binaria'] = 0
df_ipc['clase_binaria'] = np.where(df_ipc['clase_ternaria'] == 'CONTINUA', 0, 1)

df_cpe['clase_binaria'] = 0
df_cpe['clase_binaria'] = np.where(df_cpe['clase_ternaria'] == 'CONTINUA', 0, 1)

In [ ]:
# meses_train = [201906, 201907, 201908, 201909, 201910, 201911, 201912,
#                202001, 202002, 202003, 202004, 202005, 202006,
#                202007, 202008, 202009, 202010, 202011, 202012,
#                202101, 202102, 202103, 202104, 202105, 202106]

# df_ipc = df_ipc[df_ipc['foto_mes'].isin(meses_train)]
# df_cpe = df_cpe[df_cpe['foto_mes'].isin(meses_train)]

In [ ]:
print(df_ipc.shape)
print(df_cpe.shape)

(3773357, 679)

In [ ]:
# Subsampleo df_ipc:

df_subsampled = []

for mes, group in df_ipc.groupby("foto_mes"):

    X = group.drop(columns="clase_binaria")
    y = group["clase_binaria"]
    
    # Calculo la proporcion de bajas
    minority_proportion = y.value_counts(normalize=True).get(1, 0)
    
    # voy a incrementar esa proporcion por 10
    new_proportion = minority_proportion * 10

    # Apply RandomUnderSampler with the calculated strategy
    rus = RandomUnderSampler(sampling_strategy=new_proportion, random_state=semillas[0])
    X_res, y_res = rus.fit_resample(X, y)

    # Rebuild the resampled DataFrame for the group
    group_resampled = pd.concat([X_res, y_res], axis=1)
    group_resampled["foto_mes"] = mes  # Add back the 'foto_mes' column

    # Add the resampled DataFrame to the list
    df_subsampled.append(group_resampled)

# Concatenate all resampled groups into a single DataFrame
df_ipc = pd.concat(df_subsampled, ignore_index=True)

In [ ]:
# Subsampleo df_ipc:

df_subsampled = []

for mes, group in df_cpe.groupby("foto_mes"):

    X = group.drop(columns="clase_binaria")
    y = group["clase_binaria"]
    
    # Calculo la proporcion de bajas
    minority_proportion = y.value_counts(normalize=True).get(1, 0)
    
    # voy a incrementar esa proporcion por 10
    new_proportion = minority_proportion * 10

    # Apply RandomUnderSampler with the calculated strategy
    rus = RandomUnderSampler(sampling_strategy=new_proportion, random_state=semillas[0])
    X_res, y_res = rus.fit_resample(X, y)

    # Rebuild the resampled DataFrame for the group
    group_resampled = pd.concat([X_res, y_res], axis=1)
    group_resampled["foto_mes"] = mes  # Add back the 'foto_mes' column

    # Add the resampled DataFrame to the list
    df_subsampled.append(group_resampled)

# Concatenate all resampled groups into a single DataFrame
df_cpe = pd.concat(df_subsampled, ignore_index=True)

In [ ]:
print(df_ipc.shape)
print(df_cpe.shape)

In [ ]:
df_ipc.to_parquet('/home/eanegrin/datasets/competencia_02_inflacion_adj_ipc_undersampled_exp.parquet', index=False)
df_cpe.to_parquet('/home/eanegrin/datasets/competencia_02_inflacion_adj_cpe_undersampled_exp.parquet', index=False)

In [ ]:
!gsutil cp /home/eanegrin/datasets/competencia_02_inflacion_adj_ipc_undersampled_exp.parquet /home/eanegrin/buckets/b1/datasets/
!gsutil cp /home/eanegrin/datasets/competencia_02_inflacion_adj_cpe_undersampled_exp.parquet /home/eanegrin/buckets/b1/datasets/

Copying file:///home/eanegrin/datasets/competencia_02_fe_v01_undersampled_10_24M_altbinaria.parquet...
- [1 files][607.0 MiB/607.0 MiB]                                                
Operation completed over 1 objects/607.0 MiB.                                    
